In [ ]:
 !nvidia-smi

In [ ]:
!pip install langchain llama-cpp-python gradio huggingface_hub

In [ ]:
!huggingface-cli login

llama_cpp + huggingface_hub + gradio

In [ ]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download
import gradio as gr

In [ ]:
model_path = hf_hub_download(
  repo_id='issai/LLama-3.1-KazLLM-1.0-8B-GGUF4',
  filename='checkpoints_llama8b_031224_18900-Q4_K_M.gguf'
)

In [ ]:
llm = Llama(model_path, n_ctx=2048, n_gpu_layers=35)

In [ ]:
def response(question, history):
  formatted_prompt = f"[INST] Сен көмекшісің. Қазақ тілінде жауап бер. {question} [/INST]"
  output = llm(formatted_prompt, max_tokens=256, stop=["</s>", "[INST]"])
  return output["choices"][0]["text"]

In [ ]:
gr.ChatInterface(
    fn=response,
    type="messages"
).launch(debug=True)

using langchain

In [ ]:
!pip install langchain_community

In [ ]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [ ]:
llm = LlamaCpp(
  model_path=model_path,
  n_ctx=2048,
  temperature=0.7,
)

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
chat = ConversationChain(
  llm=llm,
  memory=memory,
  verbose=True
)

In [ ]:
def get_kazakh_prompt(question):
  return f"[INST] Сен көмекшісің. Қазақ тілінде жауап бер: {question} [/INST]"

In [ ]:
def response_langchain(question, history):
  prompt_question = get_kazakh_prompt(question)
  response = chat.predict(input=prompt_question)
  return response

In [ ]:
gr.ChatInterface(
  fn=response_langchain,
  type="messages"
).launch(debug=True)